In [2]:
import pandas as pd

In [46]:
test = pd.DataFrame([{"category": "OLDER FAMILIES", "quantity": 1},
                    {"category": "YOUNG FAMILIES", "quantity": 1},
                    {"category": "RETIREE", "quantity": 3}])

In [47]:
test["category"].replace("FAMILIES", "SINGLES/COUPLES", regex=True, inplace=True)

In [50]:
test["category"] = test["category"].astype("category")
test["category"].cat.set_categories([
        "NEW FAMILIES", 
        "YOUNG SINGLES/COUPLES",
        "MIDAGE SINGLES/COUPLES",
        "OLDER SINGLES/COUPLES",
        "RETIREES"
    ], ordered=True)
test = test.sort_values(by="category")

In [44]:
test["category"].str.split(" ").get(-1)

In [51]:
test

,category,quantity
0,OLDER SINGLES/COUPLES,1
2,RETIREE,3
1,YOUNG SINGLES/COUPLES,1


In [14]:
from bs4 import BeautifulSoup
from datetime import date, datetime, timedelta
import datetime as dt
from datetimerange import DateTimeRange
import numpy as np
import json
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.options import Options
import time


homeUrl = 'https://bath10.artifaxagora.com/room-bookings'
bookingUrl = f'{homeUrl}/venue-hire'
historyUrl = f'{homeUrl}/booking-history'
maxSlotLength = timedelta(hours=2)
username = 'secas20'
password = 'AlwaysonEdge345!'

# Room Identifiers
roomIDDict = {

    'buchan': 19,
    'room2': 18,
    'room1': 17,
    'choral': 23

}

edgeOptions = Options()
# edgeOptions.add_argument('--headless')
driver = webdriver.Edge(options=edgeOptions)
action = ActionChains(driver)

In [16]:
def sign_in(username, password):

    if driver.current_url != homeUrl:
        
        driver.get(homeUrl)
        time.sleep(1)

    signInElementList = driver.find_elements(By.LINK_TEXT, 'Sign In')

    if signInElementList:

        action.move_to_element(signInElementList[0]).click().perform()
        
        emailField = driver.find_element(By.CSS_SELECTOR, '#login_form #email')
        emailField.clear()
        emailField.send_keys(f'{username}@bath.ac.uk')

        passwordField = driver.find_element(By.CSS_SELECTOR, '#login_form #password')
        passwordField.clear()
        passwordField.send_keys(password)
        
        signInButton = driver.find_element(By.CSS_SELECTOR, '#login_form button[type="submit"]')
        action.move_to_element(signInButton).click().perform()

    while driver.find_elements(By.LINK_TEXT, 'Sign In'):

        if driver.find_elements(By.CSS_SELECTOR, 'div.sweet-alert.showSweetAlert.visible'):

            action.move_to_element(driver.find_element(By.CSS_SELECTOR, '.confirm')).click().perform()
            return 'Error'

        else:

            time.sleep(1)
        
    return 'Success'
    
def find_dates_not_booked(bookCancelled=True):

    # Dates are in date.

    if driver.current_url != historyUrl:
        
        driver.get(historyUrl)
        
    while driver.find_element(By.CSS_SELECTOR, '.dataTables_empty').text == 'Loading...':

        time.sleep(1)

    soup = BeautifulSoup(driver.execute_script('return document.documentElement.innerHTML'), 'html.parser')
    bookingList = soup.select_one('#future tbody').find_all('tr')

    dateSet = {date.today() + timedelta(days=i) for i in range(1, 7)}
    bookedDateSet = set()

    for booking in bookingList:

        bookingData = booking.find_all('td')

        try:

            bookingStatus = bookingData[-2].get_text()

        except IndexError:
        # No bookings found.

            break

        else:

            if all([bookCancelled, bookingStatus == 'Cancelled']):

                continue

            bookedDate = datetime.strptime(bookingData[2].get_text(), '%d/%m/%Y').date()
            bookedDateSet.add(bookedDate)

    return sorted(list(dateSet - bookedDateSet))

def filter_taken_slots(roomStr, dateToBook: date):

    if driver.current_url != homeUrl:
        
        driver.get(homeUrl)
        time.sleep(1)

    # Clear all previous room filters.
    if not driver.find_element(By.CSS_SELECTOR, '#ftal_room').get_attribute('aria-expanded'):
        
        locationFilterExpander = driver.find_element(By.CSS_SELECTOR, 'span[aria-labelledby="ftal_room"]')
        action.move_to_element(locationFilterExpander).click().perform()

    checkedBoxList = driver.find_elements(By.CSS_SELECTOR, f'span[role="checkbox"]:has(+ span[aria-selected="true"])')

    for checkbox in checkedBoxList:
        
        action.move_to_element(checkbox).click().perform()

    # Apply filters.
    checkboxToTick = driver.find_element(
        By.CSS_SELECTOR, 
        f'span[aria-labelledby="ftal_room[{roomIDDict[roomStr]}]"] span[role="checkbox"]'
    )
    action.move_to_element(checkboxToTick).click().perform()

    for dateID in ['#date_from', '#date_to']:

        dateField = driver.find_element(By.CSS_SELECTOR, dateID)
        dateField.clear()
        dateField.send_keys(date.strftime(dateToBook, '%d/%m/%Y'))
        
    # Check that schedule has been rendered
    noContentElement = driver.find_element(By.CSS_SELECTOR, '.sorry_no_content_text')
    
    while all([
        noContentElement.get_attribute('style') == 'display: none;',
        driver.find_elements(By.CSS_SELECTOR, '.jscroll > *') == []
    ]):
        
        time.sleep(1)

    soup = BeautifulSoup(driver.execute_script('return document.documentElement.innerHTML'), 'html.parser')
    takenSlotList = soup.select('.date_time')

    return [slot.get_text() for slot in takenSlotList]

def find_free_slots(takenStrList, bookingTimeRange: DateTimeRange):

    freeRangeList = [bookingTimeRange]
    
    for slot in takenStrList:
        
        timeList = slot.split(' – ')
        startTime = datetime.strptime(timeList[0], '%H:%M')
        endTime = datetime.strptime(timeList[1], '%H:%M')
        takenRange = DateTimeRange(startTime, endTime)
        newFreeRangeList = []

        for freeRange in freeRangeList:

            newFreeRangeList += freeRange.subtract(takenRange)
        
        freeRangeList = newFreeRangeList

    return freeRangeList

def save_schedule(scheduleName, scheduleDict):

    with open(f'bookingSchedules/{scheduleName}.json', 'r') as file:

        json.dump(scheduleDict, file)

def get_schedule(scheduleName):

    with open(f'bookingSchedules/{scheduleName}.json', 'r') as file:

        scheduleDict = json.load(file)

    return [DateTimeRange.from_range_text(timeRange) for day, timeRange in scheduleDict.items()]

In [14]:
def get_schedule_data_func(indexInt):

    def get_schedule_data(scheduleName):

        with open(f'bookingSchedules/{scheduleName}.json', 'r') as file:

            scheduleDict = json.load(file)

        return [DateTimeRange.from_range_text(dataList[indexInt]) for _, dataList in scheduleDict.items()]

    return get_schedule_data

get_schedule_list = get_schedule_data_func(0)
get_room_list = get_schedule_data_func(1)
get_mode_list = get_schedule_data_func(2)

In [4]:
with open(f'bookingSchedules/maxBookingWindow.json', 'w') as file:
    
    dataDict = {
        "mon": ["1900-01-01T10:00:00 - 1900-01-01T18:00:00", '', ''], 
        "tue": ["1900-01-01T10:00:00 - 1900-01-01T18:00:00", '', ''],  
        "wed": ["1900-01-01T10:00:00 - 1900-01-01T18:00:00", '', ''], 
        "thu": ["1900-01-01T10:00:00 - 1900-01-01T18:00:00", '', ''], 
        "fri": ["1900-01-01T10:00:00 - 1900-01-01T18:00:00", '', ''], 
        "sat": ["1900-01-01T10:00:00 - 1900-01-01T18:00:00", '', ''], 
        "sun": ["1900-01-01T10:00:00 - 1900-01-01T18:00:00", '', '']}
    json.dump(dataDict, file)

In [26]:
sign_in(username, password)

In [37]:

def get_week_details():
    # Dates are in datetime.date.

    if driver.current_url != historyUrl:
        driver.get(historyUrl)

    while driver.find_elements(By.CSS_SELECTOR, '.dataTables_empty'):
        time.sleep(1)

    soup = BeautifulSoup(driver.execute_script('return document.documentElement.innerHTML'), 'html.parser')
    bookingList = soup.select_one('#future tbody').find_all('tr')
    weekDict = {(date.today() + timedelta(days=i)): '' for i in range(0, 7)}

    for booking in bookingList:

        bookingData = booking.find_all('td')

        try:

            bookingStatus = bookingData[-2].get_text()

        except IndexError:
            # No bookings found.

            break

        else:

            if bookingStatus == 'Cancelled':

                continue

            bookedDate = datetime.strptime(bookingData[2].get_text(), '%d/%m/%Y').date()
            weekDict[bookedDate] = f'{bookingData[4].get_text()}, ' \
                                   f'{bookingData[5].get_text().replace(" (The Edge)", "")}'

    return weekDict

In [18]:
driver.quit()

In [6]:
username = 'advhab'
password = 'acbjhbac'

In [15]:
if driver.current_url != homeUrl:
        
        driver.get(homeUrl)
        time.sleep(1)

signInElementList = driver.find_elements(By.LINK_TEXT, 'Sign In')

if signInElementList:

    action.move_to_element(signInElementList[0]).click().perform()
    emailField = driver.find_element(By.CSS_SELECTOR, '#login_form #email')
    emailField.clear()
    emailField.send_keys(f'{username}@bath.ac.uk')
    
    passwordField = driver.find_element(By.CSS_SELECTOR, '#login_form #password')
    passwordField.clear()
    passwordField.send_keys(password)
    signInButton = driver.find_element(By.CSS_SELECTOR, '#login_form button[type="submit"]')
    action.move_to_element(signInButton).click().perform()
    
while driver.find_elements(By.LINK_TEXT, 'Sign In'):

    if driver.find_elements(By.CSS_SELECTOR, 'div.sweet-alert.showSweetAlert.visible'):

        action.move_to_element(driver.find_element(By.CSS_SELECTOR, '.confirm')).click().perform()
        print('Error')
        break

    else:

        time.sleep(1)
        
print('Success')

Error


KeyboardInterrupt: 

In [13]:
driver.quit()

{datetime.date(2023, 6, 26): '',
 datetime.date(2023, 6, 27): '',
 datetime.date(2023, 6, 28): '',
 datetime.date(2023, 6, 29): '',
 datetime.date(2023, 6, 30): '09:00 – 10:00, Music Practice Room 1',
 datetime.date(2023, 7, 1): '',
 datetime.date(2023, 7, 2): ''}

In [43]:
def get_booking_range(weekDict):

    dateBookedList = list(filter(lambda x: x[1] != '', weekDict.items()))
    print(dateBookedList)
    weekList = list(weekDict.keys())

    if dateBookedList:

        startDate, _ = dateBookedList[-1]
        startDate += timedelta(days=1)

    else:

        # No bookings found.
        startDate = weekList[0]

    return DateTimeRange(startDate, weekList[-1])

In [44]:
get_booking_range(weekDict)

[(datetime.date(2023, 6, 30), '09:00 – 10:00, Music Practice Room 1')]


2023-07-01T00:00:00 - 2023-07-02T00:00:00

In [45]:
driver.quit()

In [ ]:
def book_room(roomStr, bookingDate:date, bookingWindowTuple):

    driver.get(bookingUrl)
    nextButtonList = driver.find_elements(By.XPATH, '//button[@type="submit"]')

    # Choose Rehearsal(Solo).
    driver.find_element(By.XPATH, f'//input[@value="30"]').click()
    driver.find_element(By.LINK_TEXT, 'Next').click()
    time.sleep(1)

    # Choose room.
    driver.find_element(By.XPATH, f'//span[@aria-labelledby="ftal_room[{roomIDDict[roomStr]}]"]/span[2]').click()
    nextButtonList[2].click()
    time.sleep(1)

    # Enter date.
    dateField = driver.find_element(By.XPATH, '//input[@id="date_from_once"]')
    dateField.clear()
    dateField.send_keys(date.bookingDate)

    # Enter start time
    startTimeField = driver.find_element(By.XPATH, '//input[@id="between_which_times_from"]')
    startTimeField.clear()
    startTimeField.send_keys(bookingWindowTuple[0])

    # Enter end time.
    endTimeField = driver.find_element(By.XPATH, '//input[@id="between_which_times_to"]')
    endTimeField.clear()
    endTimeField.send_keys(bookingWindowTuple[1])
    nextButtonList[3].click()
    time.sleep(10)

    # Confirm booking.
    driver.find_element(By.XPATH, '//input[@name="arrangement_description"]').send_keys(username)
    driver.find_element(By.XPATH, '//input[@id="terms_checkbox_user"]').click()

    # Final submit button only appears on last page.
    driver.find_elements(By.XPATH, '//button[@type="submit"]')[-1].click()

In [ ]:
def closest_to_window_start(freeSlotList):
    
    # Times are in datetime.

    startTime, endTime = freeSlotList[0]
    
    if endTime - startTime > maxSlotLength:

        endTime = startTime + maxSlotLength

    return startTime, endTime

def closest_to_window_end(freeSlotList):

    # Times are in datetime.

    startTime, endTime = freeSlotList[-1]
    
    if endTime - startTime > maxSlotLength:
        
        startTime = endTime - maxSlotLength
        
    return startTime, endTime

In [ ]:
def filter_by_slot_length(freeSlotList):
    
    for i in range(4, 1, -1):
    
        longestSlotList = [slot for slot in freeSlotList if slot[1] - slot[0] >= i*0.5]
        
        if longestSlotList.size > 0:
            
            return longestSlotList
        
    return

In [ ]:
freeSlotList = find_free_slots('room2', takenSlotStrList, openingHoursList[dayInt])

In [ ]:
%%time 
for i in range(20): 
    
    print(i)

In [ ]:
takenRange = DateTimeRange(startTime, endTime)

# Ensures dates are constant for all times in datetime.

In [ ]:
a = []

for timeRange in openingHoursList[dayInt].subtract(takenRange):

    a += timeRange.subtract(takenRange1)
    

In [ ]:
timeList = takenSlotStrList[1].split(' – ')
startTime = datetime.strptime(timeList[0], '%H:%M')
endTime = datetime.strptime(timeList[1], '%H:%M')
takenRange1 = DateTimeRange(startTime, endTime)

In [ ]:
a

In [ ]:
np.array({1:1, 2:2, 3:3}.values())